In [ ]:
from matplotlib.pyplot import hot
import paramiko
from getpass import getpass

# Create the prompt for the password
password = getpass(prompt="Please Enter your SSH password: ")

# Hist Information
host = "192.168.6.71"
port = 22
username = "administrator"


try: 
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(host, port, username, password)
except paramiko.AuthenticationException:
    print("Authentication Failed.")

In [1]:
#Run a single Command
# command = "ps -ef"

# Runa a list of commands
commands = ["ps -ef", "netstat -an --inet", "last -adx", "cut -d: -fi /etc/passwd"]

for command in commands:
    # Get the output 
    stdin, stdout, stderr = ssh.exec_command(command)

    # Get the results from stdout
    lines = stdout.readlines()
    #print(lines)

    # Conver the list to a string
    output = ''.join(lines)

    # Header Output
    sepHeader = "" + "### BEGIN " + command + "###\n\n"
    # Output Footer
    SepFooter = "" + "### BEGIN " + command + "###\n\n"
    # Conncatinates the header, output, and footer
    cmd_output = sepHeader + output + SepFooter

    # Save the output to a file 
    with open("ssh_results.txt",'a') as f:
        f.write(cmd_output)

    #print (cmd_output)


ssh.close()

In [ ]:
# Start SFTP Session
from threading import local


sftp = ssh.open_sftp()

# List of files to download
files = ["/etc/hosts", "/etc/passwd","/var/log/auth.log"]

# Loop through the files and download them
for eachFile in files:
    # Create a list from the filename 
    x = eachFile.split("/")
    # Local path to store the files
    # Gets only the file name from the file path
    filename = x[-1]
    local_path = "remote_files/" + filename

    sftp.get(eachFile, local_path)
ssh.close()